In [1]:
# Import packages
import numpy as np
import SimpleITK as sitk
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import os

# -----------------------------
# Helpers
# -----------------------------
def load_image(path):
    img = sitk.ReadImage(path)
    return sitk.GetArrayFromImage(img), img

def is_probably_percent(u):
    """
    Heuristic: if uncertainty values look like 0-100 range rather than 0-1,
    treat as percent.
    """
    u_finite = u[np.isfinite(u)]
    if u_finite.size == 0:
        return False
    # If typical values are >1, it's very likely percent.
    return np.nanpercentile(u_finite, 95) > 1.0

# -----------------------------
# File Selection
# -----------------------------
def select_dose():
    path = filedialog.askopenfilename(
        title="Select the calibrated Dose image (NIfTI / MHD):",
        filetypes=[("NIfTI", "*.nii *.nii.gz"), ("MetaImage", "*.mhd"), ("All files", "*.*")]
    )
    if path:
        entry_dose.delete(0, tk.END)
        entry_dose.insert(0, path)

def select_unc():
    path = filedialog.askopenfilename(
        title="Select Relative Statistical Uncertainty image (NIfTI / MHD):",
        filetypes=[("NIfTI", "*.nii *.nii.gz"), ("MetaImage", "*.mhd"), ("All files", "*.*")]
    )
    if path:
        entry_unc.delete(0, tk.END)
        entry_unc.insert(0, path)

def select_voi():
    path = filedialog.askopenfilename(
        title="Select PTV VOI:",
        filetypes=[("NIfTI", "*.nii *.nii.gz"), ("MetaImage", "*.mhd")]
    )
    if path:
        entry_voi.delete(0, tk.END)
        entry_voi.insert(0, path)

# -----------------------------
# Results Display
# -----------------------------
def display_results(results):
    text_results.config(state="normal")
    text_results.delete("1.0", tk.END)
    for key, value in results.items():
        text_results.insert(tk.END, f"{key}: {value}\n")
    text_results.config(state="disabled")

# -----------------------------
# Main Calculations
# -----------------------------
def run_calculation():
    dose_path = entry_dose.get().strip()
    unc_path = entry_unc.get().strip()
    voi_path = entry_voi.get().strip()

    if not dose_path or not unc_path or not voi_path:
        messagebox.showerror("Error", "Please select Dose file, Uncertainty file, and PTV VOI file.")
        return

    # Read number of primaries
    try:
        N_current = float(entry_primaries.get())
        if N_current <= 0:
            raise ValueError
    except ValueError:
        messagebox.showerror("Error", "Total number of primaries must be a positive number.")
        return

    try:
        # Load images
        dose, _ = load_image(dose_path)
        rel_unc, _ = load_image(unc_path)
        voi, _ = load_image(voi_path)

        # Shape checks
        if dose.shape != rel_unc.shape or dose.shape != voi.shape:
            messagebox.showerror("Error", "Dose, Uncertainty, and VOI images do not match in shape!")
            return

        # Convert to float for safety
        dose = dose.astype(np.float64)
        rel_unc = rel_unc.astype(np.float64)

        # Auto-detect uncertainty units (percent or fraction)
        if is_probably_percent(rel_unc):
            rel_unc = rel_unc / 100.0
            unc_units_note = "Uncertainty auto-detected as % and converted to fraction."
        else:
            unc_units_note = "Uncertainty treated as fraction (0–1)."

        # Masks
        voi_mask = voi > 0
        n_vox_voi = int(np.count_nonzero(voi_mask))
        if n_vox_voi == 0:
            messagebox.showerror("Error", "VOI contains no voxels.")
            return

        dose_threshold = 80.0  # Gy
        high_dose_mask = voi_mask & (dose > dose_threshold)
        n_vox_high = int(np.count_nonzero(high_dose_mask))
        if n_vox_high == 0:
            messagebox.showinfo("Result", "No voxels in VOI with Dose > 80 Gy.")
            return

        # --------------------------
        # Mean absorbed doses
        # --------------------------
        mean_dose_voi = float(np.mean(dose[voi_mask]))
        mean_dose_high = float(np.mean(dose[high_dose_mask]))

        # ==========================================================
        # mean statistical uncertainty SŪ in voxels with Dk > 80 Gy
        # computed as RMS of voxel relative uncertainties:
        #    where s_k is the voxel RELATIVE uncertainty (fraction)
        # ==========================================================
        s_k = rel_unc[high_dose_mask]  # relative uncertainty per voxel (fraction)
        SUbar_fraction = float(np.sqrt(np.mean(s_k ** 2)))  # RMS
        SUbar_percent = SUbar_fraction * 100.0

        # ==========================================================
        #    Uncertainty (SD) of the MEAN dose in that region (Gy and %)
        #    using voxel absolute uncertainties sigma_i = D_i * u_i
        #    and propagation for a mean:
        # ==========================================================
        D_i = dose[high_dose_mask]
        sigma_i = D_i * s_k  # absolute voxel uncertainty (Gy)
        sigma_mean_Gy = float(np.sqrt(np.sum(sigma_i ** 2)) / n_vox_high)
        sigma_mean_rel_percent = (sigma_mean_Gy / mean_dose_high) * 100.0 if mean_dose_high != 0 else 0.0

        fraction_high = n_vox_high / n_vox_voi

        # ==================================================
        # MC scaling to reach target mean SU < 2%
        # ==================================================
        TARGET_SU = 2.0  # %
        current_su = SUbar_percent  

        if current_su < TARGET_SU:
            M = 1.0
            runs_required = 1
            N_required = N_current
            scaling_text = f"Target achieved: SŪ(D>80Gy) = {current_su:.2f}% (< {TARGET_SU:.2f}%)."
        else:
            M = (current_su / TARGET_SU) ** 2
            runs_required = int(np.ceil(M))
            # If each additional run uses N_current primaries, total primaries ~ runs_required * N_current
            N_required_total = runs_required * N_current
            expected_after_combination = current_su / np.sqrt(runs_required)

            scaling_text = (
                f"To reach SŪ(D>80Gy) < {TARGET_SU:.2f}%:\n"
                f"  Required multiplier M = {M:.3f}\n"
                f"  If you repeat runs with the same primaries per run:\n"
                f"    Runs needed ≈ {runs_required}\n"
                f"    Total primaries ≈ {N_required_total:.2e}\n"
                f"    Expected SŪ after combining ≈ {expected_after_combination:.2f}%"
            )
            N_required = N_required_total

        # --------------------------
        # Results
        # --------------------------
        results = {
            "VOI": os.path.basename(voi_path),
            "Uncertainty input note": unc_units_note,
            "Dose threshold (Gy)": f"{dose_threshold:.1f}",
            "Mean Absorbed Dose PTV (Gy)": f"{mean_dose_voi:.2f}",
            "Mean Absorbed Dose PTV (Dose > 80 Gy) (Gy)": f"{mean_dose_high:.2f}",
            "Total PTV Voxels": n_vox_voi,
            "Total PTV Voxels (Dose > 80 Gy)": n_vox_high,
            "Fraction of VOI voxels > 80 Gy": f"{fraction_high:.3f}",

            # This is the thesis/IAEA-style  metric on voxels with doses superior to 80 Gy
            "Mean Statistical Uncertainty SŪ in voxels D>80Gy (%)": f"{SUbar_percent:.3f}",

            # This is uncertainty of the MEAN dose (different metric) - overall dose
            "Abs. statistical uncertainty of MEAN dose (Gy)": f"{sigma_mean_Gy:.6f}",
            "Rel. statistical uncertainty of MEAN dose (%)": f"{sigma_mean_rel_percent:.3f}",

            "Current MC primaries per run": f"{N_current:.2e}",
            "Target SŪ (%)": f"{TARGET_SU:.2f}",
            "MC-GATE Simulation Runs Summary": scaling_text,
        }

        display_results(results)

    except Exception as e:
        messagebox.showerror("Error", str(e))


# ==========================================================
# GUI LAYOUT
# ==========================================================
root = tk.Tk()
root.title("PTV Dose > 80 Gy Statistical Uncertainty Calculator")
root.geometry("980x620")
root.resizable(True, True)

# Input frame
frame_inputs = ttk.LabelFrame(root, text="Input Files", padding=10)
frame_inputs.pack(fill="x", padx=10, pady=10)

ttk.Label(frame_inputs, text="Select the calibrated Dose image (NIfTI / MHD):").grid(row=0, column=0, sticky="w", pady=5)
entry_dose = ttk.Entry(frame_inputs, width=60)
entry_dose.grid(row=0, column=1, padx=5)
ttk.Button(frame_inputs, text="Browse", command=select_dose).grid(row=0, column=2)

ttk.Label(frame_inputs, text="Select Relative Statistical Uncertainty image (NIfTI / MHD):").grid(row=1, column=0, sticky="w", pady=5)
entry_unc = ttk.Entry(frame_inputs, width=60)
entry_unc.grid(row=1, column=1, padx=5)
ttk.Button(frame_inputs, text="Browse", command=select_unc).grid(row=1, column=2)

ttk.Label(frame_inputs, text="Select PTV VOI:").grid(row=2, column=0, sticky="w", pady=5)
entry_voi = ttk.Entry(frame_inputs, width=60)
entry_voi.grid(row=2, column=1, padx=5)
ttk.Button(frame_inputs, text="Browse", command=select_voi).grid(row=2, column=2)

ttk.Label(frame_inputs, text="MC primaries per run:").grid(row=3, column=0, sticky="w", pady=5)
entry_primaries = ttk.Entry(frame_inputs, width=25)
entry_primaries.grid(row=3, column=1, sticky="w", padx=5)
entry_primaries.insert(0, "1.18e8")

# Compute button
tk.Button(
    root,
    text="Compute PTV Dose > 80 Gy SU",
    command=run_calculation,
    bg="#2e8b57",
    fg="white",
    activebackground="#276749",
    activeforeground="white",
    font=("Arial", 11, "bold"),
    height=2
).pack(pady=10)

# Results frame
frame_results = ttk.LabelFrame(root, text="Results", padding=10)
frame_results.pack(fill="both", expand=True, padx=10, pady=10)

text_results = tk.Text(frame_results, height=14, wrap="word", font=("Courier", 11))
text_results.pack(fill="both", expand=True)
text_results.config(state="disabled")

root.mainloop()
 